In [1]:
from bs4 import BeautifulSoup
import requests, json, re, tqdm, os, sys, multiprocessing
from collections import defaultdict
import numpy as np
import pandas as pd
import gensim, cython
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

In [2]:
req = requests.get("https://data.nola.gov/api/views/jsrk-e98x/rows.json?accessType=DOWNLOAD")

In [3]:
req

<Response [200]>

In [4]:
html = BeautifulSoup(req.text)

In [5]:
with open(r"C:\Users\iseedeadpeople\audio\data-maining text.txt", "w", encoding="utf8") as file:
    file.write(str(req.text))

In [6]:
with open(r"C:\Users\iseedeadpeople\audio\data-maining html.txt", "w", encoding="utf8") as file:
    file.write(str(html))

In [7]:
vhod = req.text.replace("\\n", " ").replace("\xa0", " ").replace("\xad", " ").lower()

In [8]:
result = re.findall(r'"([^"]{100,})"', vhod)

In [9]:
for q in range(10):
    print(result[q])
    print(" ")

this data comes from https://nola.nextrequest.com/ and is a feed of all public records requests that have been published. some requests may not be included because they contain sensitive or personally identifiable information.
 
...               re:     louisiana public records request                           the guste iii new affordable housing units project                           new orleans, louisiana       to whom it may concern:   this letter is a request for information from the city of new orleans, department of safety and permits (“dsp”) pursuant to the louisiana public records act, § 44:1 et seq. this request pertains to information related to the guste iii: new affordable housing units project located at 1302 simon bolivar avenue, new orleans, louisiana and bounded by south robertson street, clio street, simon bolivar avenue and thalia street (the “project”). the project is being developed by the housing authority of new orleans (“hano”).       in accordance with the lo

In [10]:
#создание матрицы для сингулярного разложения, для датафрэйма нехватает памяти
dic = {}
for sent in result:
    words = sent.split()
    for w in words:
        if w not in dic:
            dic[w] = {}
        for w2 in words:
            dic[w][w2]=1

In [11]:
#df = pd.DataFrame(dic)
#df.fillna(0, inplace=True)

In [12]:
#pca = PCA()
#res = pca.fit_transform(df)

In [13]:
#разложение текста в словарь, создание списка(колво текстов) списков из кортежей (1-номер слова в словаре,2-колво повторений в тексте)
tokenized_texts = [list(gensim.utils.tokenize(t, to_lower=True)) for t in result]
dictionary = gensim.corpora.Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

In [14]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1)],
 [(0, 3),
  (1, 3),
  (2, 14),
  (3, 1),
  (11, 1),
  (16, 2),
  (17, 2),
  (18, 1),
  (21, 1),
  (22, 12),
  (23, 7),
  (25, 3),
  (27, 4),
  (28, 1),
  (31, 1),
  (32, 1),
  (33, 2),
  (34, 1),
  (35, 2),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 3),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 2),
  (46, 2),
  (47, 1),
  (48, 3),
  (49, 2),
  (50, 1),
  (51, 1),
  (52, 2),
  (53, 2),
  (54, 2),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 3),
  (71, 3),
  (72, 2),
  (73, 1),
  (74, 2),

In [15]:
len(dictionary)

11306

In [16]:
for q in range(1111):
    print(dictionary[q])

a
all
and
be
because
been
com
comes
contain
data
feed
from
have
https
identifiable
included
information
is
may
nextrequest
nola
not
of
or
personally
public
published
records
requests
sensitive
some
that
they
this
accordance
act
addresses
affordable
agreements
alleged
any
applicable
are
at
authority
avenue
b
being
between
bolivar
bounded
but
by
c
certificates
city
clayton
clio
communications
completion
concern
contracts
control
copies
correspondence
custody
d
delays
department
developed
documents
dsp
e
each
electronic
et
evidencing
f
files
final
following
for
g
gas
green
guste
h
hano
housing
i
iii
in
including
inspection
inspections
issued
it
j
kennedy
letter
letters
limited
located
louisiana
mails
mark
new
occupancy
occupy
orleans
other
otherwise
paper
partial
party
patrick
permission
permits
pertains
photos
possession
project
pursuant
r
re
regulations
related
reports
request
respectfully
robertson
safety
seq
service
simon
smith
south
street
substantial
tags
thalia
the
things
to
topics

residential
right
rising
selling
smaller
spate
spring
state


In [17]:
# LSI method

In [18]:
lsi = gensim.models.lsimodel.LsiModel(corpus=corpus, id2word=dictionary,num_topics=20, onepass=False, power_iters=30)

In [19]:
lsi.projection.u

array([[ 1.11907091e-01, -4.72047779e-02,  9.31157520e-02, ...,
        -1.06646014e-01, -3.36343314e-01,  1.47708255e-01],
       [ 1.02366150e-01,  6.47201063e-02, -1.43296579e-01, ...,
        -1.89139620e-01,  1.64105461e-01, -9.92550553e-02],
       [ 3.11858651e-01,  1.12892688e-02, -3.39347632e-01, ...,
         4.50257256e-02, -2.85255512e-02, -2.78140747e-03],
       ...,
       [ 1.79006708e-05,  2.66717724e-05,  6.09360127e-05, ...,
        -2.32714425e-04, -3.28184486e-05,  3.46917821e-04],
       [ 1.64999310e-05,  3.01103384e-05,  4.96794496e-05, ...,
        -1.56571790e-04, -8.86909316e-05,  3.24951748e-04],
       [ 1.64999310e-05,  3.01103384e-05,  4.96794496e-05, ...,
        -1.56571790e-04, -8.86909316e-05,  3.24951748e-04]])

In [20]:
lsi.projection.u.shape

(11306, 20)

In [21]:
# LDA method

In [22]:
tokenized_texts = [list(gensim.utils.tokenize(t, to_lower=True)) for t in result]
dictionary = gensim.corpora.Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

In [23]:
lda = gensim.models.ldamulticore.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=20, passes=5, random_state=42)

In [24]:
lda.show_topics()

[(16,
  '0.047*"the" + 0.038*"to" + 0.034*"and" + 0.027*"or" + 0.021*"of" + 0.016*"any" + 0.016*"a" + 0.015*"bid" + 0.014*"all" + 0.011*"i"'),
 (11,
  '0.049*"the" + 0.043*"of" + 0.034*"and" + 0.031*"or" + 0.029*"any" + 0.023*"to" + 0.016*"for" + 0.014*"records" + 0.013*"property" + 0.011*"new"'),
 (6,
  '0.051*"the" + 0.031*"of" + 0.029*"to" + 0.028*"and" + 0.022*"i" + 0.022*"for" + 0.015*"request" + 0.014*"in" + 0.013*"that" + 0.012*"all"'),
 (13,
  '0.040*"the" + 0.034*"to" + 0.027*"for" + 0.023*"and" + 0.018*"street" + 0.018*"of" + 0.017*"all" + 0.015*"orleans" + 0.015*"new" + 0.014*"since"'),
 (12,
  '0.072*"the" + 0.047*"of" + 0.036*"and" + 0.031*"to" + 0.016*"all" + 0.015*"for" + 0.014*"city" + 0.014*"or" + 0.014*"a" + 0.012*"any"'),
 (4,
  '0.033*"and" + 0.029*"or" + 0.023*"any" + 0.020*"the" + 0.019*"to" + 0.016*"property" + 0.015*"for" + 0.015*"environmental" + 0.014*"of" + 0.013*"at"'),
 (1,
  '0.077*"tax" + 0.071*"the" + 0.054*"sale" + 0.051*"of" + 0.040*"to" + 0.032*"and" 

In [25]:
# WORD2VEC method

In [26]:
train_corpus =  [TaggedDocument(doc, [i]) for i, doc in enumerate(tokenized_texts)]    
train_corpus

[TaggedDocument(words=['this', 'data', 'comes', 'from', 'https', 'nola', 'nextrequest', 'com', 'and', 'is', 'a', 'feed', 'of', 'all', 'public', 'records', 'requests', 'that', 'have', 'been', 'published', 'some', 'requests', 'may', 'not', 'be', 'included', 'because', 'they', 'contain', 'sensitive', 'or', 'personally', 'identifiable', 'information'], tags=[0]),
 TaggedDocument(words=['re', 'louisiana', 'public', 'records', 'request', 'the', 'guste', 'iii', 'new', 'affordable', 'housing', 'units', 'project', 'new', 'orleans', 'louisiana', 'to', 'whom', 'it', 'may', 'concern', 'this', 'letter', 'is', 'a', 'request', 'for', 'information', 'from', 'the', 'city', 'of', 'new', 'orleans', 'department', 'of', 'safety', 'and', 'permits', 'dsp', 'pursuant', 'to', 'the', 'louisiana', 'public', 'records', 'act', 'et', 'seq', 'this', 'request', 'pertains', 'to', 'information', 'related', 'to', 'the', 'guste', 'iii', 'new', 'affordable', 'housing', 'units', 'project', 'located', 'at', 'simon', 'boliva

In [27]:
d2v = Doc2Vec(vector_size=3, min_count=1, workers=multiprocessing.cpu_count())
d2v.build_vocab(train_corpus)
d2v.train(train_corpus, total_examples=5, epochs=5)

C:\Users\iseedeadpeople\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [28]:
d2v.docvecs[0]

array([-0.19700341,  0.03512095,  0.10492807], dtype=float32)

In [29]:
d2v.most_similar('or')

C:\Users\iseedeadpeople\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('author', 0.9999980926513672),
 ('be', 0.9999949932098389),
 ('and', 0.9999849796295166),
 ('information', 0.999869167804718),
 ('curley', 0.9998443722724915),
 ('in', 0.9997968077659607),
 ('chartres', 0.9997904896736145),
 ('stirlingprop', 0.999775767326355),
 ('that', 0.9997479915618896),
 ('untested', 0.9997413754463196)]

In [31]:
d2v.wv.most_similar('and')

[('or', 0.9999849796295166),
 ('author', 0.9999804496765137),
 ('be', 0.9999749064445496),
 ('curley', 0.9999260306358337),
 ('information', 0.9998883605003357),
 ('that', 0.9998460412025452),
 ('records', 0.9998427033424377),
 ('untested', 0.9998348355293274),
 ('to', 0.9997504949569702),
 ('chartres', 0.999728262424469)]